In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical

In [3]:
# 데이터 로드 및 전처리
df = pd.read_csv('C:/dataset/abalone.csv')

In [4]:
df

,id,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
0,0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...,...
4172,4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [5]:
# 분류를 위한 Rings 카테고리화
def categorize_rings(rings):
    if rings <= 8:
        return 0    # 어린 전복
    elif rings <= 12:
        return 1    # 중간 전복
    else:
        return 2    # 성숙 전복

df['Rings_category'] = df['Rings'].apply(categorize_rings)

In [6]:
# Sex 원-핫 인코딩
df = pd.get_dummies(df, columns=['Sex'])

In [7]:
# 특성과 타겟 분리
X = df.drop(['Rings', 'Rings_category', 'id'], axis=1)
y = df['Rings_category']

In [8]:
# 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
# CNN 입력을 위한 reshape
X_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

In [10]:
# 원-핫 인코딩
y = to_categorical(y)

In [11]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

In [12]:
# CNN 분류 모델
model_clf = Sequential([
    Conv1D(64, 2, activation='relu', input_shape=(X_train.shape[1], 1)),
    BatchNormalization(),
    MaxPooling1D(2),
    
    Conv1D(128, 2, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(2),
    
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

C:\Users\김경환\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model_clf.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [1]:
# 모델 학습
history_clf = model_clf.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

C:\Users\김경환\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.6159 - loss: 0.9313 - val_accuracy: 0.6233 - val_loss: 0.9918
Epoch 2/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6563 - loss: 0.7801 - val_accuracy: 0.6756 - val_loss: 0.9034
Epoch 3/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6769 - loss: 0.7483 - val_accuracy: 0.6697 - val_loss: 0.8378
Epoch 4/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6823 - loss: 0.7406 - val_accuracy: 0.6622 - val_loss: 0.7796
Epoch 5/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7007 - loss: 0.7180 - val_accuracy: 0.6667 - val_loss: 0.7505
Epoch 6/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6900 - loss: 0.7263 - val_accuracy: 0.6517 - val_loss: 0.7604
Epoch 7/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6882 - loss: 0.7083 - val_accuracy: 0.6906 - val_loss: 0.7073
Epoch 8/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6863 - loss: 0.7088 - val_accuracy: 0.